## Competição do Kaggle

### Nome da Competição: Google Brain - Ventilator Pressure Prediction 
#### ( Simulando um ventilador conectado ao pulmão de um paciente sedado )

### Métodos Utilizados para submeter as predições: 
### - Knn Neighbors Regressor
### - Linear Regression
### - Rede Neural Convolucional - CNN 
### - Rede Neural usando a camadaTemporal do Keras - TCN (Temporal Convolutional Network) .

### Informações:
#### Nesta competição, você simulará um ventilador conectado ao pulmão de um paciente sedado. As melhores apresentações levarão em consideração a conformidade e resistência dos atributos pulmonares.

#### A competição será pontuada como o erro absoluto médio entre as pressões previstas e reais durante a fase inspiratória de cada respiração. A fase expiratória não é pontuada. A pontuação é dada por: [X-Y], onde X é o vetor da pressão prevista e Y é o vetor das pressões reais em todas as respirações no conjunto de teste.


#### Para cada id no conjunto de teste, você deve prever um valor para a variável de pressão. O arquivo deve conter um cabeçalho e ter o seguinte formato:

#### id, pressure
#### 1,20
#### 2,23
#### 3,24
#### etc..

#### Cada série temporal representa uma respiração de aproximadamente 3 segundos. Os arquivos são organizados de forma que cada linha seja um intervalo de tempo em uma respiração e forneça os dois sinais de controle, a pressão das vias aéreas resultante e os atributos relevantes do pulmão, descritos a seguir.

#### Colunas
id - identificador de intervalo de tempo globalmente exclusivo em um arquivo inteiro.

breath_id - intervalo de tempo globalmente único para respirações.

R - atributo do pulmão indicando o quão restrita as vias aéreas são (em cmH2O / L / S). Fisicamente, é a mudança na pressão por mudança no fluxo (volume de ar por tempo). Intuitivamente, pode-se imaginar explodir um balão por um canudo. Podemos mudar R mudando o diâmetro do canudo, com R mais alto sendo mais difícil de soprar.

C - atributo do pulmão que indica o quão complacente o pulmão é (em mL / cmH2O). Fisicamente, esta é a mudança no volume por mudança na pressão. Intuitivamente, pode-se imaginar o mesmo exemplo de balão. Podemos mudar C mudando a espessura do látex do balão, com C mais alto tendo látex mais fino e mais fácil de soprar.

time_step - o carimbo de hora real.

u_in - a entrada de controle para a válvula solenóide inspiratória. Varia de 0 a 100.

u_out - a entrada de controle para a válvula solenóide exploratória. 0 ou 1.

pressão - a pressão das vias aéreas medida no circuito respiratório, medida em cmH2O.

In [ ]:
# Bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import max_error
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Carregando os Arquivos

#TREINO
dataset = pd.read_csv('train.csv')

#TESTE
testset = pd.read_csv('test.csv')

#EXEMPLO DE SUBMISSÃO
submission = pd.read_csv('sample_submission.csv')

In [ ]:
dataset.shape, testset.shape, submission.shape

In [ ]:
#dataset de treino
dataset.head(5)

In [ ]:
# dataset de teste
testset.head(5)

In [ ]:
#visualizar estatisticas básicas do dataset de treino:  média, percentagens, desvio padrão, min e máximo.
dataset.describe()

In [ ]:
dataset.head(5)

### Preparando as bases de treino e teste para usar com o KNN Regressor (Regressão)

In [ ]:
#rotulando a saída: coluna "Pressure" da base de treino
label = dataset.iloc[0: , 7]  
label

In [ ]:
#pegando todas as colunas no dataset de treino
data = dataset.iloc[0: , [0, 1, 2, 3, 4, 5, 6, 7]]
data.head(5)

In [ ]:
#pegando todas os dados do dataset de Teste
testdat = testset.iloc[0: , [0, 1, 2, 3, 4, 5, 6]]
testdat.head(5)

In [ ]:
#criando uma coluna de Pressure no dataset de test, e preenchendo com 0, para ser usada na previsão final.
testdat.insert(len(testdat.columns), 'pressure', 0)
testdat.info()

In [ ]:
#preenchendo com 0 os dados faltantes no dataset de treino
data = (data.fillna(0))

#preenchendo com 0 os dados faltantes no dataset de teste
testdat = testdat.fillna(0)

In [ ]:
data.head(3)

In [ ]:
testdat.head(3)

In [ ]:
# Splitando os dados de treino em 70% e de teste em 30%
X_train, X_test, y_train, y_test = train_test_split(data, label, random_state = 0, train_size = 0.7, )

In [ ]:
# Visualizando como está a distribuição das colunas do conjunto de treinamento.

sns.pairplot(X_train[["R", "C", "time_step", "u_in","u_out","pressure"]], diag_kind="kde")

#### Normalizando os dados Splitados - StandardScaler

In [ ]:
# Passando StandardScaler - Normalização dos dados.

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform (X_test)

In [ ]:
X_train

In [ ]:
X_test

### Método através da Regressão, basedo em k-nearest neighbors, usando o " KNeighborsRegressor ", os dados são de natureza contínua

In [ ]:
# fazendo o apreendizado com o KNN Regressor
# Regressão baseada em K vizinhos mais próximos.

# não funcionou o classifier!
# Knn = KNeighborsClassifier() é usado apenas para classificação, necesserário usar o KNeighborsRegressor() para regressão.

# usando o Knn para regressão e ajustando os dados com o FIT com os dados de treino
# usando knn=2, obteve um melhor resultado nas métricas de validação, do que usar Knn = 3,4,5,7

Knn = KNeighborsRegressor(n_neighbors=2, weights='distance')
Knn.fit(X_train, y_train)

#### Resultados com outros valores de Knn,  n_neighbors:
##### Knn = 2, ERRO MÁXIMO: 4,72, com  weights='distance'
##### kNN = 2 , ERRO MÁXIMO: 4,78, com Weights = 'default'
##### kNN = 3 , ERRO MÁXIMO: 5,34
##### kNN = 4 , ERRO MÁXIMO: 5,64
##### kNN = 5 , ERRO MÁXIMO: 6,04
##### kNN = 7 , ERRO MÁXIMO: 6,62




##### Melhor valor encontrado de Knn , foi de n_neighbors = 2 e com weights='distance'
##### Para vizinhos mais próximos, o resultado é melhor usando esse parâmetro "weights='distance "

In [ ]:
# Plotando os valores de erro máximo calculado, para cada valor de n_neighbours testados

# valores de n_nrighbors
x = [2, 3, 4, 5, 7]
# valores do erro máximo calculado
y = [4.72, 5.34, 5.64,6.04, 6.62]
#gráfico:
plt.plot(x,y, 'ro-')
plt.ylabel('Erro máximo residual calculado')
plt.xlabel('Valores de n_neighbors testados no modelo KNeighborsRegressor')
plt.grid(True)
plt.show()

#### Apartír do gráfico acima, nota-se que quanto maior o valor de n_Neigbours testado no modelo, maior o valor do erro máximo residual calculado (max_error). A função max_error, calcula o erro residual máximo, uma métrica que captura o pior caso de erro entre o valor previsto e o valor verdadeiro. Quanto menor o valor, melhor o modelo.

In [ ]:
#fazendo a predição encima dos dados de teste -> X_test

predicao = Knn.predict(X_test)
print(predicao)

#### Métricas de Regressão, para a avaliação do modelo, usando o Knn Regressor

In [ ]:
# R2-Score Coeficiente de determinação - O melhor valor de R2-SCORE possível é 1, e pode ser negativo também.

R2score = r2_score(predicao,y_test)
print('R2_Score da predição é:', R2score)

In [ ]:
# Perda da regressão, do erro absoluto mediano. - A saída de erro absoluto médio é um ponto flutuante não negativo. 
# O melhor valor possível é 0,0.

erro_mediano_absoluto = median_absolute_error(predicao,y_test)
print ('O erro mediano absoluto da regressão é:',erro_mediano_absoluto )

In [ ]:
#erro médio absoluto entre as previsões de saída e base de teste (pressão atual)
#O melhor valor possível é 0,0.

erro_medio_absoluto = mean_absolute_error(predicao,y_test)
print('O MAE, erro médio absoluto é:', erro_medio_absoluto)

In [ ]:
#erro médio quadrático entre as previsões de saída e base de teste (pressão atual)
#O melhor valor possível é 0,0.

erro_medio_quadratico = mean_squared_error(predicao,y_test)
print('O RMSE, erro médio quadratico é:',erro_medio_quadratico)

In [ ]:
# Erro máximo calculado na predição, melhor valor possível é 0.0
# A função max_error calcula o erro residual máximo, uma métrica que captura o pior
# caso de erro entre o valor previsto e o valor verdadeiro.

erro_maximo = max_error(predicao,y_test)
print('O erro máximo residual calculado foi de :',erro_maximo )

In [ ]:
#R2 score do modelo, quanto mais próximo de 1, melhor se ajusta ao modelo treinado

r2Score_modelo = Knn.score(X_train, y_train)
print('O R2score, o coeficiente de determinação do modelo foi de:', r2Score_modelo)

In [ ]:
# variancia, o melhor valor possível é 1. Os valores muito abaixo de 1 não são bons. Quanto mais próximo de 1, melhor o modelo.

variancia = explained_variance_score(y_test, predicao)
print('A Variancia do modelo é:', variancia)

#### Prevendo as pressões finais

In [ ]:
# Fazendo a predição final,prevendo as pressões finais.

resultado_final = Knn.predict(testdat)
print(resultado_final)

#### Preparando o arquivo de saída para submissão e criando o arquivo final "submisison.csv"

In [ ]:
# construindo o dataframe final, como coluna 0 ["id"] e coluna 1 ["pressure"]
index = [dataset['pressure']]
df_saida = pd.DataFrame(data = resultado_final, index = testdat['id'], columns = ['pressure'])

#passando o dataframe para um arquivo csv
df_saida.to_csv('./submission.csv', header=True)
print('./submission.csv')

In [ ]:
#data frame final para submissão
df_saida.head(10)

In [ ]:
#tamanho da base esperada, de acordo com a competição requisitou (4024000 de colunas).
len(df_saida)

In [ ]:
#informações estatísticas do data frame final
df_saida.describe()

### Usando o método Regressão Linear - LinearRegression

In [ ]:
# Carregando o modelo
regresssao_linear = LinearRegression()

In [ ]:
# Treinando o modelo
regresssao_linear.fit(X_train, y_train)

#### Métricas para a validação da Regressão Linear

In [ ]:
#predição de x_test

y_pred_regre = regresssao_linear.predict(X_test)
print('resposta da predição é:', y_pred_regre)

In [ ]:
# R2-Score Coeficiente de determinação - O melhor valor de R2-SCORE possível é 1, e pode ser negativo também.

R2score = r2_score(y_pred_regre,y_test)
print('R2_Score da predição é:', R2score)

In [ ]:
# Perda da regressão, do erro absoluto mediano. - A saída de erro absoluto médio é um ponto flutuante não negativo. 
# O melhor valor possível é 0,0.

erro_mediano_absoluto = median_absolute_error(y_pred_regre,y_test)
print ('O erro mediano absoluto da regressão é:',erro_mediano_absoluto )

In [ ]:
#erro médio absoluto entre as previsões de saída e base de teste (pressão atual)
#O melhor valor possível é 0,0.

erro_medio_absoluto = mean_absolute_error(y_pred_regre,y_test)
print('O MAE, erro médio absoluto é:', erro_medio_absoluto)

In [ ]:
#erro médio quadrático entre as previsões de saída e base de teste (pressão atual)
#O melhor valor possível é 0,0.

erro_medio_quadratico = mean_squared_error(y_pred_regre,y_test)
print('O RMSE, erro médio quadratico é:',erro_medio_quadratico)

In [ ]:
# Erro máximo calculado na predição, melhor valor possível é 0.0
# A função max_error calcula o erro residual máximo, uma métrica que captura o pior
# caso de erro entre o valor previsto e o valor verdadeiro.

erro_maximo = max_error(y_pred_regre,y_test)
print('O erro máximo residual calculado foi de :',erro_maximo )

In [ ]:
#R2 score do modelo, quanto mais próximo de 1, melhor se ajusta ao modelo treinado

r2Score_modelo = regresssao_linear.score(X_train, y_train)
print('O R2score, o coeficiente de determinação do modelo foi de:', r2Score_modelo)

In [ ]:
# variancia, o melhor valor possível é 1. Os valores muito abaixo de 1 não são bons. Quanto mais próximo de 1, melhor o modelo.

variancia = explained_variance_score(y_test, y_pred_regre)
print('A Variancia do modelo é:', variancia)

In [ ]:
# Fazendo a predição final,prevendo as pressões finais.
# Saídas da previsão, resultado das pressões - regression lINEAR

resultado_final_regressao = regresssao_linear.predict(testdat)
print(resultado_final_regressao)

#### Preparando o arquivo de saída para submissão e criando o arquivo final "submisison2.csv":

In [ ]:
# construindo o dataframe final como coluna 0 ["id"] e coluna 1 ["pressure"]
index = [dataset['pressure']]
df_saida2 = pd.DataFrame(data = resultado_final_regressao, index = testdat['id'], columns = ['pressure'])

#passando o dataframe para um arquivo csv
df_saida2.to_csv('./submission2.csv', header=True)
print('./submission2.csv')

In [ ]:
df_saida2

In [ ]:
#tamanho da base esperada, de acordo com a competição requisitou (4024000 de colunas).
len(df_saida2)

In [ ]:
#informações estatísticas do data frame final
df_saida2.describe()

#### O melhor resultado na pontuação da competição do LeaderBoard, foi usando o Liner Regression

### Usando Rede Neural - CNN -  para Regressão:

In [ ]:
print(X_train.shape)

In [ ]:
print(y_train.shape)

In [ ]:
#sequênciando o modelo
modelo = Sequential()

# Adição da primeira camada e primeira camada escondida
modelo.add(Dense(500, input_dim=8, activation='relu'))

# Adição da segunda camada escondida
modelo.add(Dense(100, activation='relu'))

# Adição da terceira camada escondida
modelo.add(Dense(50, activation='relu'))

# Adição da camada de dropout
tf.keras.layers.Dropout(0.5)

#Camada de saída
modelo.add(Dense(1))

# Compile model
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])

In [ ]:
# treinando o modelo e pegando o histórico 
historico = modelo.fit(X_train, y_train, batch_size = 128, epochs = 30,  verbose=1)

In [ ]:
# descrição do modelo
modelo.summary()

In [ ]:
print(historico.history.keys())

In [ ]:
# pLot do histórico do erro loss, durante o treinamento do modelo.

plt.plot(historico.history['mae'])
plt.title('Histórico de treinamento do Modelo - Mae')
plt.ylabel('mae')
plt.xlabel('epoch')
plt.legend(['mae'], loc='upper left')
plt.show()

In [ ]:
# previsão de x_test
y_pred_rede = modelo.predict(X_test)
print(y_pred_rede)

#### Métricas de Regressão para validar a Rede Neural - CNN

In [ ]:
# R2-Score Coeficiente de determinação - O melhor valor de R2-SCORE possível é 1, e pode ser negativo também.

R2score = r2_score(y_pred_rede,y_test)
print('R2_Score da predição é:', R2score)

In [ ]:
# Perda da regressão, do erro absoluto mediano. - A saída de erro absoluto médio é um ponto flutuante não negativo. 
# O melhor valor possível é 0,0.

erro_mediano_absoluto = median_absolute_error(y_pred_rede,y_test)
print ('O erro mediano absoluto da regressão é:',erro_mediano_absoluto )

In [ ]:
#erro médio absoluto entre as previsões de saída e base de teste (pressão atual)
#O melhor valor possível é 0,0.

erro_medio_absoluto = mean_absolute_error(y_pred_rede,y_test)
print('O MAE, erro médio absoluto é:', erro_medio_absoluto)

In [ ]:
#erro médio quadrático entre as previsões de saída e base de teste (pressão atual)
#O melhor valor possível é 0,0.

erro_medio_quadratico = mean_squared_error(y_pred_rede,y_test)
print('O RMSE, erro médio quadratico é:',erro_medio_quadratico)

In [ ]:
# Erro máximo calculado na predição, melhor valor possível é 0.0
# A função max_error calcula o erro residual máximo, uma métrica que captura o pior
# caso de erro entre o valor previsto e o valor verdadeiro.

erro_maximo = max_error(y_pred_rede,y_test)
print('O erro máximo residual calculado foi de :',erro_maximo )

In [ ]:
# variancia, o melhor valor possível é 1. Os valores muito abaixo de 1 não são bons. Quanto mais próximo de 1, melhor o modelo.

variancia = explained_variance_score(y_test, y_pred_rede)
print('A Variancia do modelo é:', variancia)

In [ ]:
# Fazendo a predição final,prevendo as pressões finais.
# Saídas da previsão, resultado das pressões - REDE NEURAL

y_pred_rede_final = modelo.predict(testdat)
print(y_pred_rede_final)

In [ ]:
len(y_pred_rede_final)

#### Preparando o arquivo de saída para submissão e criando o arquivo final "submisison3.csv":

In [ ]:
# construindo o dataframe final como coluna 0 ["id"] e coluna 1 ["pressure"]
index = [dataset['pressure']]
df_saida3 = pd.DataFrame(data = y_pred_rede_final, index = testdat['id'], columns = ['pressure'])

#passando o dataframe para um arquivo csv
df_saida3.to_csv('./submission3.csv', header=True)
print('./submission3.csv')

In [ ]:
#tamanho da base esperada, de acordo com a competição requisitou (4024000 de colunas).
len(df_saida3)

In [ ]:
#informações estatísticas do data frame final
df_saida3.describe()

### Usando o Método knn Neighbors Regressor com os parâmetros Default

In [ ]:
Knn2 = KNeighborsRegressor(n_neighbors=5)
Knn2.fit(X_train, y_train)

In [ ]:
#fazendo a predição encima dos dados de teste -> X_test

predicao2 = Knn2.predict(X_test)
print(predicao2)

In [ ]:
# R2-Score Coeficiente de determinação - O melhor valor de R2-SCORE possível é 1, e pode ser negativo também.

R2score = r2_score(predicao2,y_test)
print('R2_Score da predição é:', R2score)

In [ ]:
# Erro máximo calculado na predição, melhor valor possível é 0.0
# A função max_error calcula o erro residual máximo, uma métrica que captura o pior
# caso de erro entre o valor previsto e o valor verdadeiro.

erro_maximo = max_error(predicao2,y_test)
print('O erro máximo residual calculado foi de :',erro_maximo )

In [ ]:
# variancia, o melhor valor possível é 1. Os valores muito abaixo de 1 não são bons. Quanto mais próximo de 1, melhor o modelo.

variancia = explained_variance_score(y_test, predicao2)
print('A Variancia do modelo é:', variancia)

In [ ]:
# Fazendo a predição final,prevendo as pressões finais.

resultado_final2 = Knn2.predict(testdat)
print(resultado_final2)

#### Preparando o arquivo de saída para submissão e criando o arquivo final "submisison4.csv":

In [ ]:
# construindo o dataframe final, como coluna 0 ["id"] e coluna 1 ["pressure"]
index = [dataset['pressure']]
df_saida4 = pd.DataFrame(data = resultado_final2, index = testdat['id'], columns = ['pressure'])

#passando o dataframe para um arquivo csv
df_saida4.to_csv('./submission4.csv', header=True)
print('./submission4.csv')

In [ ]:
df_saida4

### Usando a Rede Neural, com a camada temporal convolucional do Keras - TCN (Temporal Convolutional Network)


In [ ]:
# Carregando os Arquivos

#TREINO
train_data = pd.read_csv('train.csv')

#TESTE
test_data = pd.read_csv('test.csv')

#EXEMPLO DE SUBMISSÃO
submission5 = pd.read_csv('sample_submission.csv')

In [ ]:
# rotulando a saída, as pressões.
targets = train_data[['pressure']].to_numpy().reshape(-1, 80)

# Dropando as colunas sem necessidade da base treino e da base teste
train_data.drop(['pressure', 'id', 'breath_id', 'u_out'], axis=1, inplace=True)
test_data =  test_data.drop(['id', 'breath_id', 'u_out'], axis=1)

In [ ]:
#Normalizando os dados
from sklearn.preprocessing import RobustScaler

RS = RobustScaler()
train_data = RS.fit_transform(train_data)
test_data  = RS.transform(test_data)

In [ ]:
n_features = train_data.shape[-1]

train_data = train_data.reshape(-1, 80, n_features)
test_data  = test_data.reshape(-1, 80, n_features)

n_epochs = 50
n_splits =  5

In [ ]:
X_train = train_data
y_train = targets

In [ ]:
#importando a camada TCN (Temporal Convolutional Network do Keras)

from tcn import TCN, tcn_full_summary

model = keras.models.Sequential([
    TCN(input_shape=(80, n_features), nb_filters=64, return_sequences=True, dilations=[1, 2, 4, 8, 16, 32]),
    keras.layers.Dense(1)
])
    
model.compile(optimizer="adam", loss="mae",metrics=keras.metrics.MeanAbsoluteError())


# Parâmetros da rede:

# Input shape:
# 3D tensor with shape (batch_size, timesteps, input_dim). timesteps can be None. 

# nb_filters: Presente em qualquer arquitetura ConvNet. 
# Ele está vinculado ao poder preditivo do modelo e afeta o tamanho da sua rede. 
# Quanto mais, melhor, a menos que você comece a se ajustar demais.

# dilations:controla a profundidade da camada TCN.
# Normalmente, considere uma lista com vários valores pulando de dois em dois.

# Ref:. https://pypi.org/project/keras-tcn/

In [ ]:
# Treinamento e histórico da rede
history = model.fit(X_train, y_train,  epochs=30, batch_size=1024, verbose =1)

In [ ]:
# descrição do modelo
model.summary()

In [ ]:
# mostrando os parâmetros de erros da rede
print(history.history.keys())

In [ ]:
# pLot do histórico do erro loss, durante o treinamento do modelo.

plt.plot(history.history['loss'])
plt.title('Erro do Modelo - Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
# Fazendo a predição final,prevendo as pressões finais.
# Saídas da previsão, resultado das pressões - REDE NEURAL

predict_final = model.predict(test_data)
print(predict_final)

In [ ]:
# predição final para o arquivo de submissão. 
test_preds = []
test_preds.append(model.predict(test_data).squeeze().reshape(-1, 1).squeeze())

#### Preparando o arquivo de saída para submissão e criando o arquivo final "submisison5.csv":

In [ ]:
# submissão
submission5["pressure"] = sum(test_preds)/n_splits
submission5.to_csv('submission5.csv', index=False)

In [ ]:
submission5.head(10)

In [ ]:
#informações estatísticas do data frame final
submission5.describe()

In [ ]:
len(submission5)

### Conclusões:

#### Os melhores valores encontrados nas métricas de validação para regressão, usando: R2-Score, Erro Médio Absoluto, Erro Mediano Absoluto, Erro Médio Quadrático, Erro Máximo e Variância, foi usar o modelo Linear Regression no dataset, obtendo assim também o melhor resultado no Leader Board da competição do Kaggle. Apesar dos bons valores das métricas de regressão para ambos os modelos, a melhor pontuação no LeaderBoard foi para a submissão do modelo Linear Regression. Melhor do que ter usado o dataset nos modelos Knn Neighbours Regressor , Rede Neural CNN e a camada TCN (Temporal Convolutional Network do Keras)

### Ranking dos modelos testados pela competição do Kaggle:

#### 1° "Linear Regression ()"
#### 2° k-nearest neighbors, usando o " KNeighborsRegressor () "
#### 3° Rede Neural, com a camada temporal convolucional do Keras - "TCN" (Temporal Convolutional Network)
#### 4° Rede Neural Convolucional, "CNN", para regressão.